In [1]:
import google.generativeai as genai
import os
from pathlib import Path
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load environment variables and configure API
load_dotenv()
api_key = os.environ.get("API_KEY")
if not api_key:
    raise ValueError("API_KEY environment variable not set")
genai.configure(api_key=api_key)

# Initialize the image generation model
image_generation_model = genai.GenerativeModel('gemini-1.5-flash')

def generate_custom_prompt(image_keyword: str = 'random keyword', num_image_prompts: int = 10) -> str:
    """
    Generate a custom prompt for image generation based on a keyword.
    
    Args:
        image_keyword (str): The keyword to base the prompts on.
        num_image_prompts (int): The number of prompts to generate.
    
    Returns:
        str: The generated custom prompt.
    """
    # Enhanced prompt template with more specific instructions
    prompt_template = f"""
    AI image prompt expert, create {num_image_prompts} diverse and detailed prompts based on "{image_keyword}".
    Consider various styles (2D, 3D, photorealistic), compositions, and market trends.
    For each prompt:
    1. Describe the scene in detail (colors, positioning, lighting, background, camera angle)
    2. Specify any unique elements or creative twists
    3. Suggest a mood or atmosphere
    4. Include relevant technical aspects (e.g., rendering style, art technique)

    Format: Number. Detailed prompt (no titles, 1-2 sentences each)
    """
    return prompt_template

def remove_text_before_delimiter(text: str, delimiter: str) -> str:
    """Remove text before a specified delimiter."""
    return text.split(delimiter, 1)[1] if delimiter in text else text

def generate_and_save_excel_prompts(image_keyword: str, num_image_prompts: int) -> Path:
    """
    Generate prompts and save them to an Excel file.
    
    Args:
        image_keyword (str): The keyword to base the prompts on.
        num_image_prompts (int): The number of prompts to generate.
    
    Returns:
        Path: The path to the saved Excel file.
    """
    prompt_custom = generate_custom_prompt(image_keyword, num_image_prompts)
    response = image_generation_model.generate_content(prompt_custom)
    cleaned_response = remove_text_before_delimiter(response.text, ":\n\n")
    output_lines = cleaned_response.replace("*", "").split("\n")

    df = pd.DataFrame(output_lines, columns=["Prompt"])
    output_dir = Path("output")
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / f"{image_keyword}_prompts.xlsx"
    df.to_excel(output_file, index=False)

    logging.info(f'Output saved to "{output_file}"')
    return output_file

def format_excel_file(original_input_file_path: Path, formatted_output_file_path: Path) -> None:
    """
    Format the Excel file with generated prompts.
    
    Args:
        original_input_file_path (Path): Path to the original Excel file.
        formatted_output_file_path (Path): Path to save the formatted Excel file.
    """
    df = pd.read_excel(original_input_file_path)
    
    # Improved data cleaning and formatting
    df['text_prompt'] = df['Prompt'].apply(lambda x: x if isinstance(x, str) and x.strip() and x[0].isdigit() else np.nan)
    df['text_prompt'] = df['text_prompt'].str.replace(r'^\d+\.\s*', '', regex=True)
    df = df[df['text_prompt'].notna()].rename(columns={'text_prompt': 'prompt'}).reset_index(drop=True)
    df['index'] = df.index + 1
    df.set_index('index', inplace=True)

    # Split into three sheets of 6 prompts each
    with pd.ExcelWriter(formatted_output_file_path, engine='openpyxl') as writer:
        for i in range(3):
            sheet_df = df.iloc[i*6:(i+1)*6]
            sheet_df.to_excel(writer, sheet_name=f'Sheet{i+1}', index=True)
    
    logging.info(f'Saved formatted file to {formatted_output_file_path}')

if __name__ == "__main__":
    image_keyword = "Laos"
    num_image_prompts = 18
    original_input_file_path = generate_and_save_excel_prompts(image_keyword, num_image_prompts)
    formatted_output_file_path = Path("D:/Ai_leonado/day23") / f"{image_keyword}_prompts.xlsx"
    format_excel_file(original_input_file_path, formatted_output_file_path)


D:\Ai_leonado\Script_py\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-25 17:04:36,798 - INFO - Output saved to "output\Laos_prompts.xlsx"
2024-07-25 17:04:36,986 - INFO - Saved formatted file to D:\Ai_leonado\day23\Laos_prompts.xlsx


In [4]:
import getpass
import os
from pathlib import Path
from time import sleep
import random
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, WebDriverException

# Constants
WAIT_TIME = 10
MIN_TYPE_DELAY = 0.05
MAX_TYPE_DELAY = 0.2
PROMPT_PROCESSING_DELAY = 15

def emulate_typing(element, text, min_delay=MIN_TYPE_DELAY, max_delay=MAX_TYPE_DELAY):
    """
    Emulate human-like typing in a web element with improved randomness.
    """
    for char in text:
        element.send_keys(char)
        time.sleep(random.uniform(min_delay, max_delay))
    
    # Add a random pause after typing to mimic human behavior
    time.sleep(random.uniform(0.3, 0.7))

def get_chrome_options():
    """
    Configure and return Chrome options for WebDriver with additional settings.
    """
    options = Options()
    options.add_experimental_option("detach", True)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("useAutomationExtension", False)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-save-password-bubble")
    options.add_argument("--start-maximized")  # Start with maximized window
    options.add_argument("--disable-extensions")  # Disable extensions for better performance
    return options

def open_chrome_with_profile(profile_path):
    """
    Open Chrome with a specific user profile and enhanced error handling.
    """
    chrome_options = get_chrome_options()
    chrome_options.add_argument(f"--user-data-dir={profile_path}")
    try:
        driver = webdriver.Chrome(options=chrome_options)
        return driver
    except WebDriverException as e:
        print(f"Error initializing WebDriver: {e}")
        return None

def open_leonardo_ai_profile(profile_number):
    """
    Open Leonardo AI with a specific Chrome profile and navigate to the image generation page.
    """
    profile_path = Path(os.path.expanduser("~")) / "AppData" / "Local" / "Google" / "Chrome" / "User Data" / f"Profile {profile_number}"
    driver = open_chrome_with_profile(str(profile_path))
    if driver:
        driver.get("https://app.leonardo.ai/image-generation")
        return driver
    return None

def process_prompts(driver, excel_file, sheet_name):
    """
    Read prompts from an excel sheet and interact with the Leonardo AI prompt input field.
    """
    df = pd.read_excel(io=excel_file, sheet_name=sheet_name, index_col=0)
    wait = WebDriverWait(driver, WAIT_TIME)

    try:
        label_prompt = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prompt-input"]')))
    except TimeoutException:
        print("Timeout waiting for prompt input field. Check if the page loaded correctly.")
        return

    for index, text in enumerate(df.prompt, 1):
        print(f"Processing prompt {index}/{len(df)}: {text}")
        label_prompt.click()
        label_prompt.send_keys(Keys.CONTROL + 'a' + Keys.BACKSPACE)  # Clear prompt
        emulate_typing(label_prompt, text)
        sleep(random.uniform(1.5, 2.5))  # Random delay before submitting
        label_prompt.send_keys(Keys.ENTER)
        
        # Dynamic wait time based on prompt length
        # wait_time = max(PROMPT_PROCESSING_DELAY, len(text) * 0.5)
        sleep(PROMPT_PROCESSING_DELAY)

def main():
    """
    Main function to define profile number, read excel file and call processing function.
    """
    profile_number = 1
    excel_file_path = Path(r"D:\Ai_leonado\day23\Laos_prompts.xlsx")
    sheet_name = f"Sheet{profile_number}"

    driver = open_leonardo_ai_profile(profile_number)
    if not driver:
        print("Failed to initialize WebDriver. Exiting.")
        return

    try:
        process_prompts(driver, excel_file_path, sheet_name)
    except Exception as e:
        print(f"Error encountered: {e}")
    finally:
        print("Closing browser...")
        sleep(5)
        driver.quit()

if __name__ == "__main__":
    main()


Processing prompt 1/6: Golden Hour at Kuang Si Falls: A mesmerizing view of Kuang Si Falls bathed in the warm glow of golden hour. The cascading water creates a blur of turquoise against the lush green jungle backdrop. Sunbeams pierce through the foliage, casting long shadows on the rocks. Focus on the water's movement and the vibrant colors of the surrounding nature. Render in a photorealistic style with a soft, dreamy filter. Mood: Tranquility, wonder.
Processing prompt 2/6: Street Food Stall in Luang Prabang: A bustling scene with a street food vendor grilling skewers of Laotian meat. The vendor's hands move deftly, their face illuminated by the flames. The stall is filled with vibrant spices and colorful ingredients. The background shows a lively street with locals and tourists mingling. Use a high-contrast, film grain aesthetic. Mood: Bustling, authentic, delicious.
Processing prompt 3/6: Ancient Wat Xieng Thong: A majestic view of Wat Xieng Thong, bathed in the soft morning light